# Section3a - Target and Features preparation
We will start by deciding which features we want to use in the machine learning prediction of the severity of the accident for each user.
We'll also do a final clean up of the features to remove bad entries.
The new dataframe with the target and wanted features will be stored in MySQL database to be used later on in Section3b.

**The target** data is the gravity column in the Users dataframe.

**The features** useful in the machine learning to predict the gravity of the accident are:
- *From Users dataframe*
 - Location in vehicle
 - User type
 - Sexe
 - Age
 - Journey type
 - Safety gear type
 - Safety gear worn
 - Pedestrian location (Too many entries set to "not recorded" => not used)
 - Pedestrian action (Too many entries set to "not recorded" => not used)
- *From characteristics dataframe*
 - Luminosity
 - In city
 - Intersect type
 - Weather
 - Collision type
- *From Locations dataframe*
 - Road type
 - Traffic mode
 - Nb Lanes
 - Road profil
 - Road surface
 - Road width (Too many entries set to "not recorded" => not used)
 - Installations (Too many entries set to "not recorded" => not used)
 - Location
- *From Vehicles dataframe*
 - Vehicle type
 - Fixed object hit
 - Moving obj hit
 - Impact location
 - Maneuver
- *Engineered variables*
 - day number in the year (To take into account the season)
 - Max weight differential (Between the vehicle of the user and the heaviest vehicle in the accident, or the weight of the pedestrian and the weight of the vehicle that hit them)
 


In [1]:
import pandas as pd
import numpy as np
# Provides better color palettes
import seaborn as sns

from pandas import DataFrame,Series

import matplotlib as mpl
import matplotlib.pyplot as plt
# Command to display the plots in the iPython Notebook
%matplotlib inline
import matplotlib.patches as mpatches

mpl.style.use('seaborn-whitegrid')
plt.style.use('seaborn-talk')
# Extract the list of colors from this style for later use
cycl = mpl.rcParams['axes.prop_cycle']
colors = cycl.by_key()['color']

In [2]:
from CSVtoSQLconverter import load_sql_engine
sqlEngine = load_sql_engine()

## Target and Features from the Users dataframe
We start with the Users dataframe because each entry in our features dataframe will correspond to one road user involved in an accident and have the target.

In [3]:
users_df = pd.read_sql_query('SELECT * FROM safer_roads.users',
                            sqlEngine)
users_df.head()

,accident id,pedestrian action,user type,pedestrian company,severity,pedestrian location,vehicle id,location in vehicle,safety gear type,safety gear worn,sex,journey type,age
0,201000000001,0.0,1,0.0,3,0.0,A01,1.0,2.0,1.0,1,5.0,34.0
1,201000000002,0.0,1,0.0,1,0.0,A01,1.0,1.0,1.0,2,5.0,27.0
2,201000000002,3.0,3,1.0,3,4.0,A01,NaN,NaN,NaN,1,0.0,45.0
3,201000000003,0.0,1,0.0,3,0.0,A01,1.0,2.0,3.0,1,5.0,31.0
4,201000000003,0.0,1,0.0,1,0.0,C03,1.0,1.0,1.0,2,5.0,51.0


In [4]:
def check_columns(dataf, columns):
    for col in columns:
        print('============ %s ============ ' % col)
        print(' Unique values:  ',dataf[col].unique())
        nbent = dataf[col].shape[0]
        nbna = dataf[col].isnull().sum()
        print(' Number of NaN:   %d/%d = %2.2f %%' % (nbna,nbent,(100.*nbna/nbent)))
        nbzeros = dataf[dataf[col]==0.][col].shape[0]
        print(' Number of zeros: %d/%d = %2.2f %%\n' % (nbzeros,nbent,(100.*nbzeros/nbent)))

In [5]:
check_columns(users_df, ['location in vehicle', 'user type', 'age', 'sex',
                       'journey type', 'safety gear worn','safety gear type',
                       'pedestrian action', 'pedestrian location','severity'])

============ location in vehicle ============ 
 Unique values:   [  1.  nan   2.   3.   5.   4.   8.   7.   9.   6.]
 Number of NaN:   63010/702243 = 8.97 %
 Number of zeros: 0/702243 = 0.00 %

============ user type ============ 
 Unique values:   [1 3 2 4]
 Number of NaN:   0/702243 = 0.00 %
 Number of zeros: 0/702243 = 0.00 %

============ age ============ 
 Unique values:   [  34.   27.   45.   31.   51.   40.   14.   47.   20.   39.   37.   21.
   48.   54.   36.    2.   33.   29.   25.   41.   23.   16.   81.   22.
   63.   49.   57.   67.   44.   76.   18.   19.   28.   50.   87.   59.
   55.   46.   43.   42.   17.   35.    9.   26.   32.   38.   77.   73.
   52.   24.   53.   62.   66.   60.   12.   13.   80.    8.    6.   64.
   61.   56.   79.   30.   65.   68.   82.   70.   78.   15.   72.   75.
   58.    7.   69.   85.   10.   84.   71.    5.   11.    1.   86.   74.
   90.    0.   83.    3.   88.   94.    4.   98.   91.   89.   93.   92.
   97.   96.   nan   95.   99.  104

A few different corrections are needed:
1. For "location in vehicle" we can replace the NaN with zeros as place holder to specify that this wasn't recorded.
1. The zeros and NaN values in the "age" column can be safely replaced by the average age since they represent very few entries.
1. The "journey type", "safety gear worn", and "safety gear type" have a category "unknown" or "other" which we can use to replace the NaN entries.
1. The "pedestrian action" and "pedestrian location" are mostly filled with zeroes indicating that they were not recorded therefore we won't use them as features.

In [6]:
# 1. Replace NaN with zeros
users_df['location in vehicle'].fillna(0, inplace=True)
# 2. Replace NaN with mean age
avg_age = users_df['age'].mean()
users_df['age'].fillna(avg_age, inplace=True)
# 3. Replace NaN with "other" or "unknown" categories
users_df['journey type'].fillna(9, inplace=True)
users_df['safety gear type'].fillna(9, inplace=True)
users_df['safety gear worn'].fillna(3, inplace=True)

In [7]:
feat_target_df = users_df[['accident id','vehicle id','severity','location in vehicle', 'user type',
                        'age', 'sex', 'journey type', 'safety gear worn','safety gear type']]
feat_target_df.head()

,accident id,vehicle id,severity,location in vehicle,user type,age,sex,journey type,safety gear worn,safety gear type
0,201000000001,A01,3,1.0,1,34.0,1,5.0,1.0,2.0
1,201000000002,A01,1,1.0,1,27.0,2,5.0,1.0,1.0
2,201000000002,A01,3,0.0,3,45.0,1,0.0,3.0,9.0
3,201000000003,A01,3,1.0,1,31.0,1,5.0,3.0,2.0
4,201000000003,C03,1,1.0,1,51.0,2,5.0,1.0,1.0


## Features from the Characteristics dataframe

In [8]:
charact_df = pd.read_sql_query('SELECT * FROM safer_roads.characteristics',
                            sqlEngine)
charact_df.head()

,accident id,luminosity,in city,intersect type,weather,collision type,city id,area id,datetime
0,201000000001,1,2,1,1.0,6.0,52,59,2010-06-12 19:30:00
1,201000000002,1,2,1,1.0,6.0,477,59,2010-08-07 10:00:00
2,201000000003,1,1,1,1.0,5.0,11,59,2010-09-11 16:00:00
3,201000000004,1,2,1,1.0,6.0,477,59,2010-09-22 16:30:00
4,201000000005,1,2,1,1.0,2.0,52,59,2010-10-25 12:15:00


In [9]:
wanted_cols = ['luminosity', 'in city', 'intersect type', 'weather', 'collision type']
check_columns(charact_df, wanted_cols)

============ luminosity ============ 
 Unique values:   [1 2 5 3 4]
 Number of NaN:   0/316854 = 0.00 %
 Number of zeros: 0/316854 = 0.00 %

============ in city ============ 
 Unique values:   [2 1]
 Number of NaN:   0/316854 = 0.00 %
 Number of zeros: 0/316854 = 0.00 %

============ intersect type ============ 
 Unique values:   [1 2 6 8 3 9 4 7 5 0]
 Number of NaN:   0/316854 = 0.00 %
 Number of zeros: 88/316854 = 0.03 %

============ weather ============ 
 Unique values:   [  1.   2.   4.   3.   6.   7.   8.   9.   5.  nan]
 Number of NaN:   48/316854 = 0.02 %
 Number of zeros: 0/316854 = 0.00 %

============ collision type ============ 
 Unique values:   [  6.   5.   2.   4.   1.   3.   7.  nan]
 Number of NaN:   9/316854 = 0.00 %
 Number of zeros: 0/316854 = 0.00 %



The intersection type has values 0 which do not correspond to anything but they concern very few instances overall. We can keep them in.

On the other hand some weather and collision type entries contain NaN. The categories 9 and 6 are for "other" respectively in weather and collision type. Since very few entries have NaN, we can put the NaN in this vague category.

In [10]:
charact_df['weather'].fillna(9,inplace=True)
charact_df['collision type'].fillna(6,inplace=True)

In [11]:
wanted_cols.append('accident id')
feat_target_df = feat_target_df.merge(charact_df[wanted_cols],
                             on=['accident id'],how='inner')
feat_target_df.head()

,accident id,vehicle id,severity,location in vehicle,user type,age,sex,journey type,safety gear worn,safety gear type,luminosity,in city,intersect type,weather,collision type
0,201000000001,A01,3,1.0,1,34.0,1,5.0,1.0,2.0,1,2,1,1.0,6.0
1,201000000002,A01,1,1.0,1,27.0,2,5.0,1.0,1.0,1,2,1,1.0,6.0
2,201000000002,A01,3,0.0,3,45.0,1,0.0,3.0,9.0,1,2,1,1.0,6.0
3,201000000003,A01,3,1.0,1,31.0,1,5.0,3.0,2.0,1,1,1,1.0,5.0
4,201000000003,C03,1,1.0,1,51.0,2,5.0,1.0,1.0,1,1,1,1.0,5.0


## Features from the Vehicles dataframe

In [12]:
vehicles_df = pd.read_sql_query('SELECT * FROM safer_roads.vehicles',
                            sqlEngine)
vehicles_df.head()

,accident id,vehicle type,nb occupants public transit,fixed obj hit,moving obj hit,impact location,maneuver,vehicle id
0,201000000001,13,0,1.0,0.0,1.0,1.0,A01
1,201000000002,4,0,0.0,1.0,1.0,1.0,A01
2,201000000003,16,0,0.0,2.0,1.0,17.0,A01
3,201000000003,4,0,0.0,9.0,3.0,23.0,C03
4,201000000003,4,0,0.0,9.0,6.0,1.0,B02


In [13]:
wanted_cols = ['vehicle type', 'fixed obj hit', 'moving obj hit',
              'impact location', 'maneuver']
check_columns(vehicles_df, wanted_cols)

============ vehicle type ============ 
 Unique values:   [13  4 16  1  5  2 10  7  9 15 21 11 14 22 12  8  3 18 24 20 19  6 17 23]
 Number of NaN:   0/538098 = 0.00 %
 Number of zeros: 0/538098 = 0.00 %

============ fixed obj hit ============ 
 Unique values:   [  1.   0.   4.  12.   6.   2.   8.  13.  15.  11.   3.  16.  10.   9.  14.
   5.   7.  nan]
 Number of NaN:   397/538098 = 0.07 %
 Number of zeros: 469692/538098 = 87.29 %

============ moving obj hit ============ 
 Unique values:   [  0.   1.   2.   9.   4.   6.   5.  nan]
 Number of NaN:   353/538098 = 0.07 %
 Number of zeros: 113809/538098 = 21.15 %

============ impact location ============ 
 Unique values:   [  1.   3.   6.   0.   2.   4.   8.   5.   7.   9.  nan]
 Number of NaN:   149/538098 = 0.03 %
 Number of zeros: 34944/538098 = 6.49 %

============ maneuver ============ 
 Unique values:   [  1.  17.  23.   0.   2.  22.  15.  11.  13.  10.  14.  19.   9.   5.  16.
  21.  20.  24.   4.   6.  18.  12.   3.   7.   8.  

Vehicle type has no NaN. For the other 4 columns a zero can be used to replace the NaN entries since it specifies "unknown" anyways, and very few entries have an NaN.

In [14]:
for col in wanted_cols[1:]:
    vehicles_df[col].fillna(0,inplace=True)

In [15]:
wanted_cols.extend(['accident id','vehicle id'])
feat_target_df = feat_target_df.merge(vehicles_df[wanted_cols],
                                on=['accident id','vehicle id'],how='inner')
feat_target_df.head()

,accident id,vehicle id,severity,location in vehicle,user type,age,sex,journey type,safety gear worn,safety gear type,luminosity,in city,intersect type,weather,collision type,vehicle type,fixed obj hit,moving obj hit,impact location,maneuver
0,201000000001,A01,3,1.0,1,34.0,1,5.0,1.0,2.0,1,2,1,1.0,6.0,13,1.0,0.0,1.0,1.0
1,201000000002,A01,1,1.0,1,27.0,2,5.0,1.0,1.0,1,2,1,1.0,6.0,4,0.0,1.0,1.0,1.0
2,201000000002,A01,3,0.0,3,45.0,1,0.0,3.0,9.0,1,2,1,1.0,6.0,4,0.0,1.0,1.0,1.0
3,201000000003,A01,3,1.0,1,31.0,1,5.0,3.0,2.0,1,1,1,1.0,5.0,16,0.0,2.0,1.0,17.0
4,201000000003,C03,1,1.0,1,51.0,2,5.0,1.0,1.0,1,1,1,1.0,5.0,4,0.0,9.0,3.0,23.0


## Features from the Locations dataframe

In [16]:
locations_df = pd.read_sql_query('SELECT * FROM safer_roads.locations',
                            sqlEngine)
locations_df.head()

,accident id,road type,traffic mode,nb lanes,reserved lane,road profil,road alignment,central reservation,road width,road surface,installations,location,school distance
0,201000000001,3,2.0,2.0,0.0,1.0,1.0,0.0,60.0,1.0,0.0,1.0,0.0
1,201000000002,3,2.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
2,201000000003,3,2.0,2.0,0.0,1.0,1.0,0.0,61.0,1.0,0.0,1.0,0.0
3,201000000004,3,2.0,2.0,0.0,1.0,1.0,0.0,68.0,1.0,0.0,1.0,0.0
4,201000000005,3,2.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,99.0


In [17]:
wanted_cols = ['road type', 'traffic mode', 'nb lanes',
              'road profil', 'road alignment','road surface',
               'road width', 'installations', 'location']
check_columns(locations_df, wanted_cols)

============ road type ============ 
 Unique values:   [3 2 9 4 6 1 5]
 Number of NaN:   0/316854 = 0.00 %
 Number of zeros: 0/316854 = 0.00 %

============ traffic mode ============ 
 Unique values:   [  2.   3.   0.   1.   4.  nan]
 Number of NaN:   523/316854 = 0.17 %
 Number of zeros: 17678/316854 = 5.58 %

============ nb lanes ============ 
 Unique values:   [  2.   0.   1.   4.   3.  nan   5.   6.  11.  10.   8.   7.  13.  12.   9.]
 Number of NaN:   794/316854 = 0.25 %
 Number of zeros: 37541/316854 = 11.85 %

============ road profil ============ 
 Unique values:   [  1.   0.   2.   3.   4.  nan]
 Number of NaN:   682/316854 = 0.22 %
 Number of zeros: 23340/316854 = 7.37 %

============ road alignment ============ 
 Unique values:   [  1.   4.   3.   0.   2.  nan]
 Number of NaN:   685/316854 = 0.22 %
 Number of zeros: 22355/316854 = 7.06 %

============ road surface ============ 
 Unique values:   [  1.   2.   5.   0.   7.   6.   9.   3.   8.   4.  nan]
 Number of NaN:   684/

For most columns we can again simply replace NaN by 0 which corresponds to an "unknown" category.

The columns "road width" and "installations" have respectively 35% and 90% of zeros so we can drop them.

In [18]:
wanted_cols.remove('road width')
wanted_cols.remove('installations')

In [19]:
for col in wanted_cols[1:]:
    locations_df[col].fillna(0,inplace=True)

In [20]:
wanted_cols.extend(['accident id'])
feat_target_df = feat_target_df.merge(locations_df[wanted_cols],
                                on=['accident id'],how='inner')
feat_target_df.head()

,accident id,vehicle id,severity,location in vehicle,user type,age,sex,journey type,safety gear worn,safety gear type,...,moving obj hit,impact location,maneuver,road type,traffic mode,nb lanes,road profil,road alignment,road surface,location
0,201000000001,A01,3,1.0,1,34.0,1,5.0,1.0,2.0,...,0.0,1.0,1.0,3,2.0,2.0,1.0,1.0,1.0,1.0
1,201000000002,A01,1,1.0,1,27.0,2,5.0,1.0,1.0,...,1.0,1.0,1.0,3,2.0,0.0,1.0,1.0,1.0,1.0
2,201000000002,A01,3,0.0,3,45.0,1,0.0,3.0,9.0,...,1.0,1.0,1.0,3,2.0,0.0,1.0,1.0,1.0,1.0
3,201000000003,A01,3,1.0,1,31.0,1,5.0,3.0,2.0,...,2.0,1.0,17.0,3,2.0,2.0,1.0,1.0,1.0,1.0
4,201000000003,C03,1,1.0,1,51.0,2,5.0,1.0,1.0,...,9.0,3.0,23.0,3,2.0,2.0,1.0,1.0,1.0,1.0


## Engineered features
From the existing features we can derive new features to improve the predicting power.

### Weight differential
The relative size of the vehicles involved in an accident has a direct impact on the gravity of the accident. A complementary piece of information would be the speed to deduce the momentum however we do not have any data on that.

When going through the dataset user by user, the vehicle information associated to each driver or passenger ('user type' = 1 or 2) correspond to their own vehicle therefore we have no information on the other vehicles in the accident. If the entry is for a pedestrian ('user type' = 3 or 4) then the associated vehicle is the vehicle that hit the pedestrian.

We will create a new column for the weight differential taking into account the two cases:
- for passengers and drivers the difference will be between their vehicle and the heaviest vehicle involved in the accident
- for pedestrian will simply take the weight of the vehicle that hit them

The mapping from 'vehicle type' to a crude estimate of the average vehicle weight in kilograms is stored in the [Mapper.py](https://github.com/hillairet/analysis-for-safer-roads/blob/master/Mapper.py) script.

In [21]:
from Mapper import Vehicle_Weights
# Frist we map all the vehicle types to the average weight
feat_target_df['weight diff'] = feat_target_df['vehicle type'].map(Vehicle_Weights)
# Then calculate the differential for drivers and passengers
mask = feat_target_df['user type'].isin([1,2])
feat_target_df.ix[mask,'weight diff'] = feat_target_df.groupby('accident id')['weight diff']\
                                        .transform(lambda x: x - x.max())

## Store features in database
We store the dataframe in Features table of the database to avoid recreating the dataframe whenever we want to try a different machine learning algorithm.

In [22]:
feat_target_df.shape

(702243, 28)

In [23]:
# chunksize is need for this big dataframe otherwise the transfer will fail
# (unless you change your settings in MariaDB)
feat_target_df.to_sql(name='ml_dataset', con=sqlEngine, if_exists = 'replace',
                   index=False, chunksize = 100)